In [1]:
!pip install torch torchvision torchaudio
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ucxsyjik
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ucxsyjik
  Resolved https://github.com/facebookresearch/detectron2.git to commit 8d85329aed8506ea3672e3e208971345973ea761
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 71.3 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Проверка версии и успешной установки
print("Detectron2 установлена и готова к работе!")

Detectron2 установлена и готова к работе!


In [2]:
!curl -L "https://universe.roboflow.com/ds/MTYIKt7Lm7?key=vmsvtxlL9O" > roboflow.zip
!unzip roboflow.zip -d ./dataset


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   1962      0 --:--:-- --:--:-- --:--:--  1958
100 98.2M  100 98.2M    0     0  10.7M      0  0:00:09  0:00:09 --:--:-- 17.2M
Archive:  roboflow.zip
replace ./dataset/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

# Зарегистрируем набор данных
register_coco_instances("traffic_train", {}, "./dataset/train/_annotations.coco.json", "./dataset/train")
register_coco_instances("traffic_val", {}, "./dataset/valid/_annotations.coco.json", "./dataset/valid")

# Проверка: загрузка метаданных
traffic_metadata = MetadataCatalog.get("traffic_train")
dataset_dicts = DatasetCatalog.get("traffic_train")
print("Данные успешно загружены!")


WARNING [11/07 15:09:49 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/07 15:09:49 d2.data.datasets.coco]: Loaded 3530 images in COCO format from ./dataset/train/_annotations.coco.json
Данные успешно загружены!


In [4]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Настраиваем конфигурацию
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("traffic_train",)
cfg.DATASETS.TEST = ("traffic_val",)  # Тестовый датасет
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Загрузка предобученных весов
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # Базовая скорость обучения
cfg.SOLVER.MAX_ITER = 1000    # Установи побольше итераций, если это необходимо
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # Точное число для R-CNN
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(traffic_metadata.thing_classes)  # Указываем количество классов

# Создаем объект тренера и запускаем обучение
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[11/07 15:09:50 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[11/07 15:09:51 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/07 15:09:58 d2.utils.events]:  eta: 0:04:31  iter: 19  total_loss: 3.182  loss_cls: 2.689  loss_box_reg: 0.4321  loss_rpn_cls: 0.01196  loss_rpn_loc: 0.006035    time: 0.2782  last_time: 0.3245  data_time: 0.0135  last_data_time: 0.0148   lr: 4.9953e-06  max_mem: 1735M
[11/07 15:10:10 d2.utils.events]:  eta: 0:04:44  iter: 39  total_loss: 3.096  loss_cls: 2.525  loss_box_reg: 0.5282  loss_rpn_cls: 0.009637  loss_rpn_loc: 0.007981    time: 0.3020  last_time: 0.3163  data_time: 0.0152  last_data_time: 0.0050   lr: 9.9902e-06  max_mem: 1735M
[11/07 15:10:16 d2.utils.events]:  eta: 0:04:32  iter: 59  total_loss: 2.749  loss_cls: 2.201  loss_box_reg: 0.4877  loss_rpn_cls: 0.01446  loss_rpn_loc: 0.01005    time: 0.2978  last_time: 0.2871  data_time: 0.0092  last_data_time: 0.0051   lr: 1.4985e-05  max_mem: 1735M
[11/07 15:10:22 d2.utils.events]:  eta: 0:04:31  iter: 79  total_loss: 2.31  loss_cls: 1.711  loss_box_reg: 0.5799  loss_rpn_cls: 0.008313  loss_rpn_loc: 0.008255    time: 0.2985

In [5]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Настройка оценщика COCO для тестового набора
evaluator = COCOEvaluator("traffic_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "traffic_val")

# Запуск оценки
print("Запуск оценки модели на валидационном наборе данных...")
inference_on_dataset(trainer.model, val_loader, evaluator)


WARNING [11/07 15:15:14 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [11/07 15:15:14 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/07 15:15:14 d2.data.datasets.coco]: Loaded 801 images in COCO format from ./dataset/valid/_annotations.coco.json
[11/07 15:15:14 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/07 15:15:14 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[11/07 15:15:14 d2.data.common]: Serializing 801 elements to byte tensors and concatenating them all ...
[11/07 15:15:14 d2.data.common]: Serialized dataset takes 0.22 MiB
Запуск оценки модели на валидационном наборе данных...
[11/07 15:15:14 d2.evaluation.evaluator]: Start inf

OrderedDict([('bbox',
              {'AP': 15.971651283577264,
               'AP50': 24.052741819544902,
               'AP75': 18.702264168466822,
               'APs': 7.997317595725155,
               'APm': 15.386152543449633,
               'APl': 17.38775586928111,
               'AP-Signs': nan,
               'AP-Green Light': 15.825587005676761,
               'AP-Red Light': 9.699444599999708,
               'AP-Speed Limit 10': nan,
               'AP-Speed Limit 100': 6.67388208137384,
               'AP-Speed Limit 110': 3.1553350661811135,
               'AP-Speed Limit 120': 13.150119876829319,
               'AP-Speed Limit 20': 22.328751755376537,
               'AP-Speed Limit 30': 15.333004290748153,
               'AP-Speed Limit 40': 31.92703300177052,
               'AP-Speed Limit 50': 4.101985666848225,
               'AP-Speed Limit 60': 12.066531815347522,
               'AP-Speed Limit 70': 12.761674719372248,
               'AP-Speed Limit 80': 6.5805268463

In [6]:
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.data import transforms as T

cfg.INPUT.MIN_SIZE_TRAIN = (640, 800)  # Увеличим минимальный размер
cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "range"

cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = True

# Добавляем аугментацию данных
augmentation = [
    T.ResizeShortestEdge(short_edge_length=(640, 800), max_size=1333, sample_style='choice'),
    T.RandomFlip(horizontal=True, vertical=False),
    T.RandomBrightness(0.8, 1.2),
    T.RandomContrast(0.8, 1.2),
    T.RandomRotation(angle=[0, 90])
]

def custom_mapper(dataset_dict):
    mapper = DatasetMapper(is_train=True, augmentations=augmentation)
    return mapper(dataset_dict)

# Задаем кастомный загрузчик данных с аугментацией
cfg.DATASETS.TRAIN = ("traffic_train",)
train_loader = build_detection_train_loader(cfg, mapper=custom_mapper)


WARNING [11/07 15:16:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/07 15:16:23 d2.data.datasets.coco]: Loaded 3530 images in COCO format from ./dataset/train/_annotations.coco.json
[11/07 15:16:23 d2.data.build]: Removed 3 images with no usable annotations. 3527 images left.
[11/07 15:16:23 d2.data.build]: Using training sampler TrainingSampler
[11/07 15:16:23 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[11/07 15:16:23 d2.data.common]: Serializing 3527 elements to byte tensors and concatenating them all ...
[11/07 15:16:23 d2.data.common]: Serialized dataset takes 0.96 MiB
[11/07 15:16:23 d2.data.build]: Making batched data loader with batch_size=2


In [7]:
cfg.MODEL.BACKBONE.NAME = "build_resnet_backbone"
cfg.MODEL.RESNETS.DEPTH = 101


In [8]:
from detectron2 import model_zoo

# Загрузка готовой конфигурации с ResNet-101
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")

# Остальные параметры остаются прежними
cfg.DATASETS.TRAIN = ("traffic_train",)
cfg.DATASETS.TEST = ("traffic_val",)
cfg.SOLVER.MAX_ITER = 1800
cfg.SOLVER.BASE_LR = 0.0001
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.INPUT.MIN_SIZE_TRAIN = (640, 800)
cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "range"


In [9]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[11/07 15:16:25 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f6e8b1.pkl: 243MB [00:02, 113MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[11/07 15:16:27 d2.engine.train_loop]: Starting training from iteration 0
[11/07 15:17:52 d2.utils.events]:  eta: 2:05:18  iter: 19  total_loss: 3.041  loss_cls: 2.684  loss_box_reg: 0.2458  loss_rpn_cls: 0.07575  loss_rpn_loc: 0.03071    time: 4.2356  last_time: 4.1048  data_time: 0.2450  last_data_time: 0.1827   lr: 1.9981e-06  max_mem: 12986M
[11/07 15:19:16 d2.utils.events]:  eta: 2:03:05  iter: 39  total_loss: 2.929  loss_cls: 2.615  loss_box_reg: 0.2403  loss_rpn_cls: 0.03701  loss_rpn_loc: 0.01904    time: 4.2262  last_time: 4.1400  data_time: 0.2296  last_data_time: 0.1863   lr: 3.9961e-06  max_mem: 12986M
[11/07 15:20:40 d2.utils.events]:  eta: 2:01:11  iter: 59  total_loss: 2.784  loss_cls: 2.427  loss_box_reg: 0.2419  loss_rpn_cls: 0.08223  loss_rpn_loc: 0.03049    time: 4.2150  last_time: 4.3534  data_time: 0.2313  last_data_time: 0.3448   lr: 5.9941e-06  max_mem: 12986M
[11/07 15:22:04 d2.utils.events]:  eta: 1:59:35  iter: 79  total_loss: 2.528  loss_cls: 2.186  loss_box_

In [10]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Настройка оценщика COCO для тестового набора
evaluator = COCOEvaluator("traffic_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "traffic_val")

# Запуск оценки
print("Запуск оценки улучшенной модели на валидационном наборе данных...")
inference_on_dataset(trainer.model, val_loader, evaluator)


WARNING [11/07 17:22:15 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [11/07 17:22:15 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/07 17:22:15 d2.data.datasets.coco]: Loaded 801 images in COCO format from ./dataset/valid/_annotations.coco.json
[11/07 17:22:15 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[11/07 17:22:15 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[11/07 17:22:15 d2.data.common]: Serializing 801 elements to byte tensors and concatenating them all ...
[11/07 17:22:15 d2.data.common]: Serialized dataset takes 0.22 MiB
Запуск оценки улучшенной модели на валидационном наборе данных...
[11/07 17:22:15 d2.evaluation.evaluator]

OrderedDict([('bbox',
              {'AP': 11.210774990233078,
               'AP50': 20.303645437335152,
               'AP75': 9.879316759358133,
               'APs': 5.402415711369002,
               'APm': 10.015589972081637,
               'APl': 12.036401020394075,
               'AP-Signs': nan,
               'AP-Green Light': 11.803586826063468,
               'AP-Red Light': 7.44387181243268,
               'AP-Speed Limit 10': nan,
               'AP-Speed Limit 100': 2.6739335885397493,
               'AP-Speed Limit 110': 0.0,
               'AP-Speed Limit 120': 3.273061914434726,
               'AP-Speed Limit 20': 21.707740495332327,
               'AP-Speed Limit 30': 11.017422136309445,
               'AP-Speed Limit 40': 17.917548142611185,
               'AP-Speed Limit 50': 3.9590987174117185,
               'AP-Speed Limit 60': 10.137718446751796,
               'AP-Speed Limit 70': 10.182235612152345,
               'AP-Speed Limit 80': 5.0289665027947095,
     

In [11]:
import torch

# Указываем путь, где будет сохранена модель
model_path = "./output/faster_rcnn_resnet50_final.pth"

# Сохраняем модель
torch.save(trainer.model.state_dict(), model_path)
print(f"Модель успешно сохранена в {model_path}")


Модель успешно сохранена в ./output/faster_rcnn_resnet50_final.pth


In [14]:
import onnxruntime as ort
import numpy as np
import cv2
import time

# Загружаем YOLOv8 модель best.onnx
ort_session = ort.InferenceSession("best.onnx")

# Подготовка изображения
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (640, 640))
    img = img.transpose(2, 0, 1)  # HWC to CHW
    img = np.expand_dims(img, axis=0).astype(np.float32)  # Добавляем batch dimension
    return img / 255.0

# Инференс для YOLO
def yolo_inference(image_path):
    img = preprocess_image(image_path)
    start_time = time.time()
    outputs = ort_session.run(None, {"images": img})
    inference_time = time.time() - start_time
    return outputs, inference_time

# Инференс для Faster R-CNN
def faster_rcnn_inference(image_path):
    img = cv2.imread(image_path)
    start_time = time.time()
    outputs = predictor(img)  # predictor загружен ранее
    inference_time = time.time() - start_time
    return outputs, inference_time


In [13]:
pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00


In [15]:
import shutil

# Создаем архив всех файлов в /content
shutil.make_archive("/content/full_project_backup", 'zip', "/content")
print("Проект успешно сохранен в /content/full_project_backup.zip")


Проект успешно сохранен в /content/full_project_backup.zip
